##Install Libraries  - Tranformers, TPU (xla)

In [2]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl
!pip install Transformers

     |████████████████████████████████| 133.6MB 94kB/s 
     |████████████████████████████████| 61kB 2.7MB/s 
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
     |████████████████████████████████| 1.8MB 6.5MB/s 
     |████████████████████████████████| 2.9MB 25.0MB/s 
     |████████████████████████████████| 890kB 40.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=cc8b99d27ee4011f3e30509285605c8a1afc4b5af3a8bec4ad962648a77b87bc
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


##Import libaries and set up TPU


In [4]:
import torch

from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

# imports the torch_xla package
import torch_xla
import torch_xla.core.xla_model as xm

# Creates a random tensor on xla:1 (a Cloud TPU core)
dev = xm.xla_device()

#Model
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = model.to(dev)
#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#Read test file from Github

import requests
url = "https://raw.githubusercontent.com/newspap/MyRepo/main/BERT_for_dummies.txt"
resp = requests.get(url)
input_ = resp.text.split("\r\n") 
input_ = list(filter(lambda x: len(x) > 5, input_))



<Response [200]>


In [7]:
from random import sample 
input_ = resp.text.split("\r\n") 
input_ = list(filter(lambda x: len(x) > 5, input_))
print(sample(input_, 10))

['Uncertainty in Deep Learning. How To Measure?', 'Diversity Sampling Cheatsheet', 'Since we were not quite successful at augmenting the dataset, now, we will rather reduce the scope of the problem. We define a binary classification task where the �flight� queries are evaluated against the remaining classes, by collapsing them into a single class called �other�. The distribution of labels in this new dataset is given below.', 'Image for post', 'towardsdatascience.com', 'Hands-on real-world examples, research, tutorials, and cutting-edge techniques delivered Monday to Thursday. Make learning your daily ritual. Take a look', 'Sign up for The Daily Pick', 'My new article provides hands-on proven PyTorch code for question answering with BERT fine-tuned on the SQuAD dataset.', 'Fellow Top Medium Writer. Fellow of Harvard University. Certified Learner. I help curious minds become AI practitioner.', 'The ultimate beginner guide for understanding, building and training GANs with bulletproof Py

##Continue ask questions

In [ ]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

def group_words(s, n):
    words = s.split()
    for i in range(0, len(words), n):
        yield ' '.join(words[i:i+n])

def get_loss(ans):
    return ans.get('loss')

while True:
    # if doc changed, reload it

    answers=[]
    question = input(bcolors.OKBLUE + "Question: " + bcolors.ENDC)

    if question == '' :
     
        break
    else:
        for longParagraph in input_:
            for paragraph in list(group_words(longParagraph, 180)):
                #encoding = tokenizer.encode_plus(text=question,text_pair=paragraph)

                #inputs = encoding['input_ids']  #Token embeddings
                #sentence_embedding = encoding['token_type_ids']  #Segment embeddings
              
                inputs = tokenizer(question, paragraph, return_tensors='pt')
                tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'].tolist()[0]) #input tokens
                start_positions = torch.tensor([1])
                end_positions = torch.tensor([3])

                #outputs = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))
                with torch.no_grad():
                    inputs = inputs.to(dev)
                    start_positions = start_positions.to(dev)
                    end_positions = end_positions.to(dev)
                    outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)
                loss = outputs.loss
                #start_scores, end_scores
                start_index = torch.argmax(outputs.start_logits)

                end_index = torch.argmax(outputs.end_logits)

                answer = ' '.join(tokens[start_index:end_index+1])

                corrected_answer = ''

                for word in answer.split():
                    
                    #If it's a subword token
                    if word[0:2] == '##':
                        corrected_answer += word[2:]
                    else:
                        corrected_answer += ' ' + word
                #if (corrected_answer.startswith(" [")):
                #    print(corrected_answer)
                lossVal = loss.item()
                
                if (len(corrected_answer) > 0):
                    print(bcolors.FAIL + str(lossVal) + ": " + bcolors.ENDC + bcolors.WARNING + corrected_answer + bcolors.ENDC) 
                    #print(paragraph)
                answers.append({"loss":lossVal, "answer":corrected_answer, "paragraph": paragraph})
    answers.sort(key=get_loss)
    print("-------------------------- Question -----------------------------------------------------")
    print(":: " + question)
    print("------------------------- Answer  ------------------------------------------------------")
    
    for ans in answers[-15:]:
        coloredAns = (bcolors.OKGREEN if ans['loss'] > 10 else bcolors.FAIL) + str(ans['loss']) + ": " + bcolors.ENDC + bcolors.WARNING + ans['answer'] + bcolors.ENDC

        print(coloredAns)
        startIdx = ans['paragraph'].lower().find(ans['answer'])

        ansLength = len(ans['answer'])

        if (startIdx != -1):
            hightlightedPara = ans['paragraph'][0:startIdx] + bcolors.WARNING + \
                    ans['paragraph'][startIdx:startIdx + ansLength] + bcolors.ENDC
            if (startIdx + ansLength < len(ans['paragraph'])):
                hightlightedPara += ans['paragraph'][startIdx + ansLength]
        else :
            hightlightedPara = ans['paragraph']
        print(hightlightedPara)

    print("############################### For Question ###############################")
    print(bcolors.HEADER +  question  + bcolors.ENDC)
    print()

Question: what is the topic
9.119222640991211:  motivation
7.770511627197266:  intent classification
8.401050567626953:  question answering
8.623769760131836:  understanding the intuition with hands - on pytorch code
5.185699462890625:  towardsdatascience . com
8.531424522399902:  ambiguous intent labeling
7.712638854980469:  transformer , especially how its attention mechanism helps in solving the intent classification task by learning contextual relationships
